In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install jiwer

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

MODEL_PATH = "/content/drive/MyDrive/train_model/small/Whisper"

# Load model và processor từ thư mục đã lưu
processor = AutoProcessor.from_pretrained(MODEL_PATH)
model = AutoModelForSpeechSeq2Seq.from_pretrained(MODEL_PATH).to("cuda:0").half()  # Dùng float16 để tối ưu GPU

# Tạo pipeline
PIPE = pipeline(
    task="automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    device="cuda:0",
    torch_dtype=torch.float16
)

PIPE_KWARGS = {"language": "vi", "task": "transcribe"}

Device set to use cuda:0


In [ ]:
import os
import torchaudio

DATASET_PATH = "/content/drive/MyDrive/train_model/Data_Goc"

def load_local_dataset(dataset_path):
    dataset = []
    for file in os.listdir(dataset_path):
        if file.endswith(".wav"):
            audio_path = os.path.join(dataset_path, file)
            text_path = audio_path.replace(".wav", ".txt")

            if os.path.exists(text_path):
                waveform, sample_rate = torchaudio.load(audio_path)
                with open(text_path, "r", encoding="utf-8") as f:
                    transcript = f.read().strip()

                dataset.append({"audio": waveform, "sentence": transcript})

    return dataset

my_data = load_local_dataset(DATASET_PATH)
print(f"Loaded {len(my_data)} samples.")

Loaded 2000 samples.


In [ ]:
from tqdm import tqdm
import numpy as np
import jiwer

@torch.inference_mode()
def predict(data):
    y_pred = []
    y_true = []

    for sample in tqdm(data, desc="Processing audio"):
        audio_tensor = sample["audio"]
        print(audio_tensor.shape)
        output = PIPE(audio_tensor.squeeze(0).cpu().numpy(), generate_kwargs={**PIPE_KWARGS, "return_timestamps": True})

        y_pred.append(output["text"])
        y_true.append(sample["sentence"])

    return y_true, y_pred

y_true, y_pred = predict(my_data)

# Tính WER

JIWER_TRANS = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemoveKaldiNonWords(),
    jiwer.RemoveMultipleSpaces(),
    jiwer.Strip(),
    jiwer.RemovePunctuation(),
    jiwer.ReduceToListOfListOfWords(),
])

wer = 100 * jiwer.wer(
    reference=y_true,
    hypothesis=y_pred,
    reference_transform=JIWER_TRANS,
    hypothesis_transform=JIWER_TRANS,
)

print(f"WER on custom dataset = {wer:.1f}%")

Processing audio:   0%|          | 0/2000 [00:00<?, ?it/s]

torch.Size([1, 74400])


Processing audio:   0%|          | 1/2000 [00:00<17:02,  1.96it/s]

torch.Size([1, 309440])


Processing audio:   0%|          | 2/2000 [00:01<32:56,  1.01it/s]

...

torch.Size([1, 196640])


Processing audio: 100%|█████████▉| 1999/2000 [28:17<00:00,  1.19it/s]

torch.Size([1, 162400])


Processing audio: 100%|██████████| 2000/2000 [28:18<00:00,  1.18it/s]


WER on custom dataset = 7.3%


WER on custom dataset = 8.9%


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.5 MB/s eta 0:00:00
